## Project Supply Chain -- First Data Exploration

**Exploration of metadata (without Text Mining technics)**
Some ideas:

- Response rate, influence of brand or source, verified_purchase or not
<br> <span style="color: blue"> We took a look at verified purchases and how the reviewers rated differently </span>

- Distribution of scores.
<br> <span style="color: blue"> We took a look at the distribution of scores in various scenarios </span>

- Influence of the marketplace or the company on the distribution of notes (hypothesis testin could be used for  this kind of analysis )
<br> <span style="color: blue"> We only have information about wether the reviewers were selected by the company or not, which we took a look at </span>

- Information about the 10 most active users, with a small analysis on it (distribution of scores, response rate, company...).
<br> <span style="color: blue"> We looked at the 10 most active users </span>

**Goal 2 Analysis of text (and cleaning if necessary ). You will need to complete the text mining module to be able to do this part.**
Some ideas :
- Analyze the punctuation according to the note

- Analyze the length of the text (nb character, nb words...) according to the note.

- Analyze the frequency of email addresses, links, phone numbers...

- Occurrence of words, wordcloud...
We did a word cloud already to have an overview and saw that we need to improve on stop-words

- N-gram

- Occurrence of some words : delivery order, return order, delivery, SAV, customer service...

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
nltk.download('stopwords')
nltk.download('punkt')

%matplotlib inline

In [ ]:
import os
project_path = os.path.abspath(os.path.join(os.getcwd(), os.path.pardir, os.path.pardir)).replace('\\', '/')
project_path

In [ ]:
data_hc = pd.read_csv(project_path + r"/data/amazon_reviews_us_Video_Games_v1_00.tsv", sep="\t", error_bad_lines=False)
data_dc = pd.read_csv(project_path + r"/data/amazon_reviews_us_Digital_Video_Games_v1_00.tsv", sep="\t", error_bad_lines=False)
data_raw = pd.concat([data_hc,data_dc], axis = 0)

### explore data

In [ ]:
data_raw.shape

In [ ]:
data_raw.info()

In [ ]:
data_raw.head()

In [ ]:
# check null data
print(data_raw.isnull().sum(axis = 0) / len(data) * 100)

In [ ]:
# drop missing data, since the amount of missing data is very low
data = data_raw.dropna(axis = 0)

#### check data quality and clean data

In [ ]:
# check duplicates
print(data["marketplace"].value_counts()) #Only data from us marketplace, so we can drop the row
print("number of duplicated customer ids = \n", len(data[data['customer_id'].duplicated() == True])) #most likely because customers ordered multiple items
print("number of duplicated review ids = \n", len(data[data['review_id'].duplicated() == True])) 
#has to be 0 in order to ensure the ID is unique, i think we can drop this row as well in this case
#placeholder for code that shows if product_id, product_title and product_parent are fully correlated
print("product_category: \n", data["product_category"].value_counts())
print("star rating: \n", data["star_rating"].value_counts())
print("vines: \n",data["vine"].value_counts())
print("verified_purchases: \n",data["verified_purchase"].value_counts())

In [ ]:
# drop the rows we do not need for this analysis or our model
to_drop = ["marketplace"] # we did not drop "review_id", "product_id", "product_parent" for now as they might 
#still be usefull going forward
data = data.drop(to_drop, axis=1)

#### Reviews per product

In [ ]:
reviews_per_product = data.groupby(["product_id"])["review_id"].nunique().reset_index(name="num_of_reviews").\
    sort_values(by=(['num_of_reviews']), ascending=False)
reviews_per_product.head()

In [ ]:
reviews_per_product["num_of_reviews"].quantile([0.01,0.1,0.25, 0.5,0.75,0.9,0.99])

#### countplot of various data

In [ ]:
sns.countplot(x=data["product_category"])

In [ ]:
sns.countplot(x=data["star_rating"])

In [ ]:
sns.countplot(x=data["verified_purchase"])
#there are a lot of non-verified purchases 
#let's look at how the rating distribution of these reviews looks like compared to the verified ones

In [ ]:
sns.countplot(x=data[data["verified_purchase"]=="Y"]["star_rating"])

In [ ]:
sns.countplot(x=data[data["verified_purchase"]=="N"]["star_rating"])
#there are a lot more lower ratings in comparison. 
#It is possible that customers were so unhappy, that they created a 2nd account just to review the game negatively again

In [ ]:
#there are definitely more hard-copy sales than digital sales, let's look at the ratings from the reviews for each one
sns.countplot(x=data[data["product_category"]=="Video Games"]["star_rating"])

In [ ]:
sns.countplot(x=data[data["product_category"]=="Digital_Video_Games"]["star_rating"])
#we can see from this simple analysis, that there are a lot more 1-star reviews for digital products

#### reviews over time

In [ ]:
reviews_over_time = data.groupby("review_date").agg({"review_id":"count"}).plot(kind="line")

In [ ]:
num_rev_prod_per_rating = data.groupby("star_rating").agg({"review_id":lambda x: x.nunique(), "product_id": lambda x: x.nunique()} )
num_rev_prod_per_rating["rev_per_prod"] = num_rev_prod_per_rating.apply(lambda x: x["review_id"] / x["product_id"], axis=1)
num_rev_prod_per_rating["rev_per_prod"].plot(kind="bar", title="Reviews per product")
#reviews per product by rating class, customers seem to review more often when they are happy

#### 10 most rated titles

In [ ]:
data["product_title"].value_counts().head(10).plot(kind="barh")

#### Top 10 titles where the reviews recieved the most votes

In [ ]:
data[["product_title", "total_votes"]].nlargest(10, ["total_votes"]).plot(x="product_title", y="total_votes", kind="barh")

#### Top 10 titles have the most 5-star reviews

In [ ]:
data[data["star_rating"] == 5]["product_title"].value_counts().head(10).plot(kind="barh")

#### Top 10 titles with the lowest rated reviews

In [ ]:
data[data["star_rating"] == 1]["product_title"].value_counts().head(10).plot(kind="barh")
#that's where SimCity went, reviewers did not like this game at all

#### Top 10 games with the most helpful reviews

In [ ]:
data[["product_title", "helpful_votes"]].nlargest(10, ["helpful_votes"]).plot(x="product_title", y="helpful_votes", kind="barh")
#Customers found reviews of SimCity the most helpful allthough the game also had a lot of bad reviews. this again shows that 
#this title was highly controversial

In [ ]:
#now we will look at the distribution of vine reviews to non-vine reviews
sns.histplot(data, x="vine", stat="percent", multiple="dodge", shrink=0.8)

In [ ]:
sns.histplot(data[data["vine"]=="Y"], x="star_rating", stat="percent", multiple="dodge", shrink=5)

In [ ]:
sns.histplot(data[data["vine"]=="N"], x="star_rating", stat="percent", multiple="dodge", shrink=10)
#we can see that the vine reviews are more likely to give 3, 4 or 5 stars compared to the non-vine reviews

### tokenization and stemming of review_body

In [ ]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.stem.snowball import EnglishStemmer
stemmer = EnglishStemmer()


stop_words.update(["car", "work", "product", "install"])

def tokenization_and_stemming(text):
    tokens = []
    # exclude stop words and tokenize the document, generate a list of string 
    for word in word_tokenize(text):
        if word.lower() not in stop_words:
            tokens.append(word.lower())

    filtered_tokens = []
    
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if token.isalpha(): # filter out non alphabet words like emoji
            filtered_tokens.append(token)
            
    # stemming
    # Removes ing also in anything ...
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return_string = " ".join(stems)
    
    return return_string

In [ ]:
df_review_body = data.dropna(subset=['review_body'])

In [ ]:
df_review_body["processed_reviews"] = df_review_body["review_body"].apply(lambda x: tokenization_and_stemming(x))

In [ ]:
df_review_body.head()

### word cloud of processed_reviews

In [ ]:
df_processed_reviews = df_review_body.dropna(subset="processed_reviews")
processed_review_string = df_processed_reviews.groupby("star_rating").aggregate({"processed_reviews":lambda x: " \n ".join(x)})

In [ ]:
def wc_for_rating(rating):
    wordcloud = WordCloud(collocations=True).generate(processed_review_string.loc[rating][0][1:5000000].replace("one", "").replace("use", "").replace(" br ", " ").replace("car", "").replace("work", ""))
    # Display the generated image:
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()

In [ ]:
wc_for_rating(1)
wc_for_rating(2)
wc_for_rating(3)
wc_for_rating(4)
wc_for_rating(5)

In [ ]:
STOP -- end of code